In [9]:
import requests
import re
import math
from collections import Counter
URL = "https://o136z8hk40.execute-api.us-east-1.amazonaws.com/dev/get-list-of-conferences"

In [10]:
confs = requests.get(url = URL).json()

In [11]:
confs.keys()

dict_keys(['display_paid', 'paid', 'free', 'display_free'])

In [12]:
len(confs['paid'])

103

In [13]:
conferences = confs['paid'] + confs['free']

### Task 1

In [51]:
for conf in conferences:
    print(f"{conf['confName']}, {conf['confStartDate']}, {conf['venue']}, {conf['entryType']}, {conf['confUrl']}\n\n")

Antarcticonf The Conference At The Edge Of The World, 05 Jan, 2020, Antarctic Peninsula (from San Antonio, Chile), Paid, https://antarcticonf.com/


Trion All India Bootcamp, 19 Jan, 2020, Delhi Technological University, Shahbad Daulatpur Village, Rohini Delhi, 110042, Paid, https://konfhub.com/trionbootcamp


DELIVERY CONF , 21 Jan, 2020, Hyatt Regency Seattle, Howell Street, Seattle, WA, USA, Paid, https://www.deliveryconf.com


AppSec California, 21 Jan, 2020, Annenberg Community Beach House, 415 Pacific Coast Highway, Santa Monica, CA 90402, United States, Paid, https://2020.appseccalifornia.org/


NDC Security 2020, 22 Jan, 2020, Clarion Hotel Oslo, Dronning Eufemias gate 15, 0191 Oslo, Norge, Paid, https://ndc-security.com/


Machine Learning Developers, 22 Jan, 2020, NIMHANS Convention center, Paid, https://www.mlds.analyticsindiasummit.com/


NDC London 2020 , 27 Jan, 2020, Queen Elizabeth II Centre, Broad Sanctuary, London, UK, Paid, https://ndc-london.com/


Distributed Syste

### Task 2

In [15]:
seen_confs = []
duplicate_conf = []
for conf in conferences:
    if seen_confs:
        if conf['confName'] in seen_confs:
            duplicate_conf.append(conf)
        seen_confs.append(conf['confName'])
    else:
        seen_confs.append(conf['confName'])

In [16]:
print(f"There are {len(duplicate_conf)} exact duplicate conferences.")

There are 6 exact duplicate conferences.


In [17]:
duplicate_conf

[{'emailId': '',
  'city': 'London',
  'twitter_handle': 'DOES_EUR',
  'user_id': '1540431802',
  'country': ' UK',
  'imageURL': '"https://storage.googleapis.com/konfhub-bd9c9.appspot.com/95575.jpg?Expires=4730767037&GoogleAccessId=firebase-adminsdk-r3qh4%40konfhub-bd9c9.iam.gserviceaccount.com&Signature=OsK1hEcZLDW5TJTsHpGbGGbZPiWiUFL7W7ZLTzaDxUS736GeAyS%2FETVA1K1TUeFayuor2WTmktzT3pJKrtZWOGV4aUIqNTOM5MbLwE4U1OiNYsVuisNHHe%2Fte5qAwNGlQfPgPb0oWXpLkRxXL%2Bx6DHe%2FBGRAUsTwJgfkrm9shu4ANsFXsYM6bjOZj26834Cv1LcpWD1bWIE9NRqOBD0mIH2DNtq8DQPizMRhXsl%2B6QQb4RjkSlO20kiud7m%2FD4m2e9ej451vYjAf1ty4yOKe3%2Ff1UiqokbXJOBOtGxMDpCM9z1oKNcmTs0pkNGpRpsjuB7PXoBWe34WkJ9u6pY%2Fqsw%3D%3D"',
  'venue': 'InterContinental London – The O2',
  'searchTerms': ' Devops, DevOps Enterprise Summit, London, June, Paid, 1577167039,  UK',
  'confName': 'DevOps Enterprise Summit',
  'state': '',
  'long': '-0.00013779999994767422',
  'confEndDate': '25 Jun, 2020',
  'conference_id': 1577167039,
  'lat': '51.5027176',
  'key

### Task 3
#### Finding Semantic Duplicates using Cosine Similarity

In [38]:


WORD = re.compile(r"\w+")


def get_cosine_similarity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def get_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [45]:
conf_score = {}
for i, conf1 in enumerate(conferences):
    for j, conf2 in enumerate(conferences):
        if i != j:
            conf1_name = get_vector(conf1['confName'])
            conf2_name = get_vector(conf2['confName'])
            conf_score[get_cosine_similarity(conf1_name, conf2_name)] = [conf1['confName'], conf2['confName']]

In [49]:
conf_score = dict(sorted(conf_score.items(), reverse=True))


### Conferences sorted in descending order of their similarity score

In [50]:
conf_score

{1.0000000000000002: ['AWS Community Day', 'AWS Community Day'],
 1.0: ['MATLAB EXPO 2020 India', 'MATLAB EXPO 2020 India'],
 0.8164965809277259: ['React Day Norway', 'React Day'],
 0.7745966692414834: ['Alexa Community Day',
  'Alexa Community Day Bengaluru 2020'],
 0.7071067811865475: ['QCon', 'QCon London'],
 0.6963106238227914: ['International Conference On Big Data IoT Cyber Security And Information Technology',
  'International Conference On Internet Of Things Big Data Analytics And Information Technology'],
 0.6666666666666667: ['Amazon Alexa Paris', 'Amazon Alexa VOICE'],
 0.6172133998483676: ['Women in Data Science Pune Conference',
  'The Global Women in Data Science Hyderabad'],
 0.5773502691896258: ['MATLAB EXPO 2020 India', 'Agile India 2020'],
 0.5477225575051661: ['Microsoft Ignite The Tour 2019 2020',
  'Microsoft Lgnite The Tour Washington'],
 0.5163977794943222: ['International Conference On Internet Of Things Big Data Analytics And Information Technology',
  'Interne